Bibliotecas usadas:

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import pickle
from sklearn.metrics import confusion_matrix, classification_report
import sklearn.metrics as metrics
from surprise import SVD, SVDpp, Dataset, accuracy, Reader, BaselineOnly, KNNBaseline
from surprise.model_selection import GridSearchCV, cross_validate
from sklearn.tree import DecisionTreeRegressor


Cria uma coluna com o numero total de ratings que o artigo teve
''' O motivo é para evitar artigos classificados com 5 estrelas, mas apenas por 1 ou alguns usuários.
Esses artigos não são relevantes se forem populares o suficiente para que mais usuários os classifiquem.'''

In [2]:
df_artigos = pd.read_csv('shared_articles.csv')
df_usuarios = pd.read_csv('users_iter.csv')

filtro_tipo_evento_view = df_usuarios["eventType"]=="VIEW"
filtro_tipo_evento_like = df_usuarios["eventType"]=="LIKE"

data = pd.merge(left=df_artigos, right=df_usuarios.loc[filtro_tipo_evento_view].drop_duplicates(['personId','contentId']), on='contentId')

num_ratings = pd.DataFrame(data.groupby('contentId').count()['rating']).reset_index()
data = pd.merge(left=data, right=num_ratings, on='contentId')
data.rename(columns={'rating_x': 'rating', 'rating_y': 'numRatings'}, inplace=True)

matrix = data.pivot_table(
    index='personId',
    columns='contentId',
    values='rating'
)

# Correlações de pares
'''
As correlações de pares revelam relações de interesse potenciais. 
A correlação de pares é calculada entre linhas ou colunas do DataFrame.
Como os valores da matriz são os ratings dos artigos, então a recomendação é relacionada diretamente ao rating,numero de rating e usuário.
pega itens similares ao que for passado usando a correlação de pearson (do metodo .corrwith que usa pearson como padrão da lib Pandas)
'''

In [3]:
def pegar_artigos_similares_correlacao_pares(movie_title, n_ratings_filter=100, n_recommendations=10):
    #print(matrix[movie_title])
    similar = matrix.corrwith(matrix[movie_title])
    corr_similar = pd.DataFrame(similar, columns=['correlacao'])
    corr_similar.dropna(inplace=True)
    
    orig = data.copy()
    
    corr_with_movie = pd.merge(
        left=corr_similar, 
        right=orig, 
        on='contentId')[['contentId', 'title', 'correlacao', 'numRatings']].drop_duplicates().reset_index(drop=True)
    
    result = corr_with_movie[corr_with_movie['numRatings'] > n_ratings_filter].sort_values(by='correlacao', ascending=False)
    
    return result.head(n_recommendations)


In [4]:
print(pegar_artigos_similares_correlacao_pares(6044362651232258738));

In [5]:
title  correlacao  numRatings
1206  Cinco competências comportamentais para você s...    1.000000         103
918        10 Modern Software Over-Engineering Mistakes    0.740744         118
535   Custo do Erro - Cinco motivos para investir em...    0.568914         121
192   Ganhe 6 meses de acesso ao Pluralsight, maior ...    0.261114         142
90    Ray Kurzweil: The world isn't getting worse - ...    0.224478         143
1364  Psicóloga de Harvard diz que as pessoas julgam...    0.177282         140
201                    Livro: Retrospectivas Divertidas    0.125102         193
1317  Um bilhão de arquivos mostram quem vence a dis...    0.110088         148
413   Former Google career coach shares a visual tri...   -0.024125         267
1204                              O chefe é gay. E daí?   -0.075072         103

Como pode ser visto no resultado, o primeiro artigo é o proprio passado como parametro e o resto a correlação vai diminuindo.

# Content-based - TF-IDF
'''
As duas funções seguintes fazem recomendação a partir do metodo content-based TF-IDF que basicamente mede a importancia das palavras mas ignora as palavras mais frequentes como
'the/a' (palavras pequenas)
'''


In [6]:
def pegar_artigos_similares_porTitulo_Tf_Idf():
    
    tf=TfidfVectorizer()
    tf_mat=tf.fit_transform(data['title'])

    linear_kernel_ = linear_kernel(tf_mat, tf_mat)

    similar=linear_kernel_[2].tolist()

    ind=[]
    val=[]
    for i in range(len(similar)):
        ind.append(i)
        val.append(similar[i])    
    
    dic={'index':ind,'value':val}

    rec_df=pd.DataFrame(dic)

    #print(rec_df)

    rec=rec_df.merge(data,on='index')   

    a=rec.sort_values(by='value',ascending=False).head(10)
    
    print(data['title'][2])
    print(a[['title', 'value']])

def pegar_artigos_similares_porConteudo_Tf_Idf():
    
    tf=TfidfVectorizer()
    tf_mat=tf.fit_transform(data['text'].fillna(''))

    cosine_sim = cosine_similarity(tf_mat)

    similar=cosine_sim[2].tolist()

    ind=[]
    val=[]
    for i in range(len(similar)):
        ind.append(i)
        val.append(similar[i])    
    
    dic={'index':ind,'value':val}

    rec_df=pd.DataFrame(dic)

    #print(rec_df)

    rec=rec_df.merge(data,on='index')   

    a=rec.sort_values(by='value',ascending=False).head(10)

    print(data['title'][2])
    print(a[['title', 'value']])


In [7]:
#colocar a coluna index e exclui duplicadas
data = data.drop_duplicates(subset='title', keep="first").reset_index()

pegar_artigos_similares_porConteudo_Tf_Idf()

In [8]:
Google Data Center 360° Tour
                                                 title     value
85   Google Is Finally Redesigning Its Biggest Cash...  0.481396
118       Practical End-to-End Testing with Protractor  0.392924
268                Building a digital-banking business  0.390531
296  App-only bank Atom just launched - here's what...  0.382130
132  Google Cloud Datastore simplifies pricing, cut...  0.372017
119  How Mature is Your Organization when it Comes ...  0.364058
17   Using Gamified Hacking Challenges To Attract N...  0.354738
201  Open Source Giant Red Hat Launches First Block...  0.352467
348   Vidente reflete sobre os desafios contemporâneos  0.351830
375       Humans Can Still Do One Thing Better Than AI  0.348447

Resultado mais satisfatorio com relação a conteudo dos artigos. É passado como parametro o titulo "Google Data Center 360° Tour" e nos resultados a maioria dos artigos tem relação com o artigo informado.

# Usando um classificador Bayesianos
'''
    Aqui basicamente vai pegar os artigos que o usuário deu 5 estrelas
    usando a estrela como label e o text (conteudo) como feature
    vamos treinar um modelo para tentar prever se o usuário vai gostar ou não de um certo artigo
    seria como uma lista de 'algo que vc talvez goste'.
    Como a base que temos é pequena (3000 artigos), a precisão da previsão é baixa (chegou no maximo em 20%)
'''

In [9]:
def pegar_artigos_similares_porConteudo_Bayesiano():

    count = CountVectorizer()

    # separando feature de target (no nosso caso titulo ou conteudo do artigo e target seria a classificação dele (rating))
    X = data['text']
    y = data['rating']

    # cria um conjunto de palavras a partir da coluna title ou text dos artigos
    palavras_transformer = count.fit(X)
    print(len(palavras_transformer.vocabulary_))
  
    # faz o transforme nos titulos
    X = palavras_transformer.transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

    sm = SMOTE(random_state=2)
    X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())

    # Agora treinar o modelo usando Naive Bais Algorithm

    nb = MultinomialNB()
    nb.fit(X_train, y_train)

    modelo_Finalizado = open("Finalized_Model_NB.pkl", "wb")
    pickle.dump(nb, modelo_Finalizado)

    # fazer predicoes no conjunto de teste
    preds = nb.predict(X_test)
        
    # Validar o modelo usando confusion matrix e classification report
    print(confusion_matrix(y_test, preds))
    print('\n')
    print(classification_report(y_test, preds))
    print(metrics.accuracy_score(y_test, preds))

In [10]:
pegar_artigos_similares_porConteudo_Bayesiano()

In [11]:
[[273 379 305 280 429 384] 
 [257 354 290 304 407 377] 
 [271 364 317 261 466 378] 
 [281 344 312 279 427 400] 
 [308 344 316 262 375 369] 
 [307 350 301 274 422 365]]


              precision    recall  f1-score   support

           0       0.16      0.13      0.15      2050
           1       0.17      0.18      0.17      1989
           2       0.17      0.15      0.16      2057
           3       0.17      0.14      0.15      2043
           4       0.15      0.19      0.17      1974
           5       0.16      0.18      0.17      2019

    accuracy                           0.16     12132
   macro avg       0.16      0.16      0.16     12132
weighted avg       0.16      0.16      0.16     12132

0.1618034948895483

Com a base de 3 mil artigos o resultado foi muito baixo, com apenas 16% de acuracia. Isso por causa que tem muito pouco artigo para fazer o treinamento.

In [12]:
df_livros = pd.read_csv('GoodReads_100k_books.csv')

df_livros['ratingAprox']  = df_livros.apply(lambda row: int(round(row.rating)) , axis = 1)

#print(df_livros[['title', 'rating', 'ratingAprox', 'totalratings']].sort_values('ratingAprox'))

def pegar_livros_similares_porConteudo_Bayesiano():
    
    count = CountVectorizer()

    # separando feature de target (no nosso caso titulo ou conteudo do artigo e target seria a classificação dele (rating))
    X = df_livros['desc'].apply(lambda x: np.str_(x))
    print(X)
    y = df_livros['ratingAprox']

    # cria um conjunto de palavras a partir da coluna title ou text dos artigos
    palavras_transformer = count.fit(X)
    print(len(palavras_transformer.vocabulary_))
  
    # faz o transforme nos titulos
    X = palavras_transformer.transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

    sm = SMOTE(random_state=2)
    X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())

    # Agora treinar o modelo usando Naive Bais Algorithm

    nb = MultinomialNB()
    nb.fit(X_train, y_train)

    modelo_Finalizado = open("Finalized_Model_NB_2.pkl", "wb")
    pickle.dump(nb, modelo_Finalizado)

    # fazer predicoes no conjunto de teste
    preds = nb.predict(X_test)
        
    # Validar o modelo usando confusion matrix e classification report
    print(confusion_matrix(y_test, preds))
    print('\n')
    print(classification_report(y_test, preds))
    print(metrics.accuracy_score(y_test, preds))

In [13]:
pegar_livros_similares_porConteudo_Bayesiano()

In [14]:
[[    2     0     0    15   449     8]
 [    0     0     0     1    12     0]
 [    1     0     0     8    92     1]
 [    6     0    39   329  3202    67]
 [   11     2    89  1059 22984   487]
 [    2     1     3    44  1016    70]]


              precision    recall  f1-score   support

           0       0.09      0.00      0.01       474
           1       0.00      0.00      0.00        13
           2       0.00      0.00      0.00       102
           3       0.23      0.09      0.13      3643
           4       0.83      0.93      0.88     24632
           5       0.11      0.06      0.08      1136

    accuracy                           0.78     30000
   macro avg       0.21      0.18      0.18     30000
weighted avg       0.71      0.78      0.74     30000

0.7795

Como pode ser visto, teve precisão de 77% para a base de dados dos livros com 100 mil livros. Mostrando que quanto maior a base melhor o resultado.

# METODOS USANDO COLLABORATIVE FILTERING
'''
 A Filtragem Colaborativa é baseada na ideia de que usuários semelhantes a mim podem ser usados ​​para prever o quanto eu gostarei de um determinado
  artigo que esses usuários visualizaram/gostaram mas eu não.
'''

In [15]:
def collaborativeFiltering_SDV(data_artigos_usuarios):
    # Ver numeros totais de usuarios e artigos
    n_users = data_artigos_usuarios['personId'].nunique()
    n_artigos = data_artigos_usuarios['contentId'].nunique()

    print('Numero de usuarios: ', n_users)
    print('Numero de Artigos: ',n_artigos)

    # Cria nova coluna id para o usuario
    unique_user_id = pd.DataFrame(data_artigos_usuarios['personId'].unique(),columns =['personId']).reset_index()
    unique_user_id['novo_id_usuario'] = unique_user_id['index']
    del unique_user_id['index']

    # cria nova coluna id para os artigos
    unique_business_id = pd.DataFrame(data_artigos_usuarios['contentId'].unique(),columns =['contentId']).reset_index()
    unique_business_id['novo_id_conteudo'] = unique_business_id['index']
    del unique_business_id['index']

    new_complete_df = data_artigos_usuarios.merge(unique_user_id,on='personId',how ='left')
    new_complete_df = new_complete_df.merge(unique_business_id,on='contentId',how ='left')

    df = new_complete_df[['novo_id_usuario','novo_id_conteudo','rating']]
    print(df.head(10))
    # Cria um objeto reader da lib surprise com escalas de rating de 0 a 5
    reader = Reader(rating_scale=(0, 5))
    # Carrega o dataset dos artigos em 5 folds
    data = Dataset.load_from_df(df,reader)
    #data.split(n_folds=5)

    # Modelando usando SVD da lib surprise
    algo = SVD()
    # Valida usando como base RMSE e MAE
    perf = cross_validate(algo, data, measures=['RMSE', 'MAE'])
    #print(perf)

    # Treina o dataset e acha os predict
    trainset = data.build_full_trainset()
    algo.fit(trainset)

    # predict o ratingo do usuario para o artigo id 0
    svd_pred = algo.predict(str('1'),str('3'),r_ui=3,verbose=True)

In [16]:
collaborativeFiltering_SDV(data)


In [17]:
{'test_rmse': array([1.78978369, 1.7766941 , 1.77971374, 1.78538199, 1.76183896]), 'fit_time': (1.5080702304840088, 1.4912209510803223, 1.5066721439361572, 1.4903011322021484, 1.5055773258209229), 'test_time': (0.06301712989807129, 0.041886091232299805, 0.041910648345947266, 0.061964988708496094, 0.0445401668548584)}
user: 9          item: 3          r_ui = 3.00   est = 2.50   {'was_impossible': False}